In [1]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import matplotlib.pyplot as plt

In [2]:
edges = [(0,1), (0,2), (1,2), (1,3)]
n_qubits = 4

In [3]:
gamma_opt = 1.50
beta_opt  = 1.49

In [4]:
%pip install pylatexenc

Note: you may need to restart the kernel to use updated packages.


In [5]:
def qaoa_circuit(n, edges, gamma, beta):
    qc = QuantumCircuit(n)

    # |+> initial state
    qc.h(range(n))

    # Cost Hamiltonian
    for i, j in edges:
        qc.cx(i, j)
        qc.rz(2 * gamma, j)
        qc.cx(i, j)

    # Mixer Hamiltonian
    for i in range(n):
        qc.rx(2 * beta, i)

    qc.measure_all()
    return qc

qc = qaoa_circuit(n_qubits, edges, gamma_opt, beta_opt)

try:
    qc.draw("mpl")
except Exception:
    # Fallback to text drawer if optional deps (e.g., pylatexenc) are missing
    print(qc.draw(output="text"))

In [6]:
# Initialize IBM Quantum Runtime service with token
service = QiskitRuntimeService(channel="ibm_quantum_platform", token="En51cKC4RBd7UK7e7zNEAXA9PRErxD5DwnZL1Mf46dO9")

qiskit_runtime_service._discover_account:WARNING:2025-12-30 21:09:52,394: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-30 21:09:57,724: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


In [7]:
backend = service.least_busy(
    operational=True,
    simulator=False,
    min_num_qubits=n_qubits
)

backend

qiskit_runtime_service.backends:WARNING:2025-12-30 21:11:34,134: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-30 21:11:36,896: Using instance: open-instance, plan: open


<IBMBackend('ibm_fez')>

In [15]:
from qiskit_ibm_runtime import Batch
from qiskit import transpile

# Use real hardware backend
backend_real = service.least_busy(
    operational=True,
    simulator=False,
    min_num_qubits=n_qubits
)

# Transpile circuit to match hardware constraints
qc_transpiled = transpile(qc, backend=backend_real, optimization_level=1)

# Create a batch execution context and run on real hardware
with Batch(backend=backend_real) as batch:
    sampler = Sampler()
    
    # Run the transpiled quantum circuit on real IBM hardware
    job = sampler.run([qc_transpiled], shots=4000)
    result = job.result()
    
print("Job completed successfully on real hardware!")

qiskit_runtime_service.backends:WARNING:2025-12-30 21:17:51,376: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-30 21:17:52,466: Using instance: open-instance, plan: open


Job completed successfully on real hardware!


In [22]:
# Extract and display measurement results
sampler_result = result[0]
data_bin = sampler_result.data

# Access the measurement BitArray and convert to counts
meas_bitarray = data_bin.meas

# Convert BitArray to dictionary of counts using get_counts method
counts_dict = meas_bitarray.get_counts()

print("Measurement Results (counts):")
for bitstring, count in sorted(counts_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"{bitstring}: {count}")

# Visualize results
if counts_dict:
    plot_histogram(counts_dict)
else:
    print("No measurement data available")

Measurement Results (counts):
1101: 344
0111: 313
0101: 291
1111: 290
0000: 262
1000: 256
1001: 248
1011: 242
0001: 241
1100: 234
0010: 233
0011: 229
1010: 222
0100: 218
0110: 191
1110: 186


In [ ]:
# Visualize the measurement results in a histogram
import matplotlib.pyplot as plt

# Create histogram using matplotlib directly
fig, ax = plt.subplots(figsize=(12, 6))

bitstrings = list(counts_dict.keys())
counts = list(counts_dict.values())

ax.bar(range(len(bitstrings)), counts, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_xlabel('Bitstring', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.set_title('QAOA Max-Cut Results on IBM Quantum Hardware', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(bitstrings)))
ax.set_xticklabels(bitstrings, rotation=45, ha='right')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
total_shots = sum(counts_dict.values())
print(f"\nTotal Shots: {total_shots}")
print(f"Unique Bitstrings: {len(counts_dict)}")
print(f"Most Common Result: {max(counts_dict.items(), key=lambda x: x[1])}")

<Figure size 640x480 with 0 Axes>


Total Shots: 4000
Unique Bitstrings: 16
Most Common Result: ('1101', 344)
